<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Final Project - Introduction to Data Science
    <div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
    @ FIT-HCMUS, VNU-HCM 📌
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 4.0 - Data modelling
</div>

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Read data

In [2]:
movie_df = pd.read_csv('./../data/processed.csv')
movie_df

,Name,Released year,Genre,Director,Runtime (h),Score,Status,Language,Budget ($),Revenue ($)
0,Fast X,2023,"'Action', 'Crime', 'Thriller'",Dan Mazeau,2.37,72.0,Released,English,340000000.0,704709660.0
1,Trolls Band Together,2023,"'Animation', 'Family', 'Music', 'Fantasy', 'Co...",Thomas Dam,1.53,72.0,Released,English,95000000.0,173800000.0
2,Robot Apocalypse,2021,"'Science Fiction', 'Action'",Marcus Friedlander,1.45,21.0,Released,English,NaN,NaN
3,Five Nights at Freddy's,2023,"'Horror', 'Mystery'",Emma Tammi,1.83,78.0,Released,English,20000000.0,286700000.0
4,Oppenheimer,2023,"'Drama', 'History'",Christopher Nolan,3.02,81.0,Released,English,100000000.0,951000000.0
...,...,...,...,...,...,...,...,...,...,...
4995,Secret in Their Eyes,2015,"'Thriller', 'Mystery', 'Drama', 'Crime'",Billy Ray,1.85,64.0,Released,English,19500000.0,34854990.0
4996,The Vatican Tapes,2015,"'Thriller', 'Horror'",Mark Neveldine,1.52,53.0,Released,English,13000000.0,1784763.0
4997,Song to Song,2017,"'Romance', 'Drama', 'Music'",Terrence Malick,2.15,55.0,Released,English,10000000.0,1710528.0
4998,Divine Intervention,2023,'Comedy',Pedro Pablo Ibarra,1.67,82.0,Released,Spanish; Castilian,NaN,NaN


## Feature engineering

In [3]:
df = movie_df[['Genre', 'Runtime (h)', 'Score', 'Language', 'Budget ($)', 'Revenue ($)']].dropna()

one_hot_df = pd.DataFrame()
def splitGenre(s):
    return s.split(', ')
genre_stack = df['Genre'].apply(splitGenre).agg(pd.Series).stack().agg(pd.Series).stack()
genre_dummy = pd.get_dummies(genre_stack).groupby(level = 0).sum()
genre_dummy.columns = ['Genre_' + col.replace("'", "") for col in genre_dummy.columns]
df = pd.concat([genre_dummy, df], axis = 1)

label_encoder=LabelEncoder()
df['Language_encoded'] = label_encoder.fit_transform(df['Language'])

df = df.drop(['Genre', 'Language'], axis=1)

profit_percentage = (df['Revenue ($)'] - df['Budget ($)']) / df['Budget ($)']

df['Success'] = df.apply(lambda row: 'Successful' if row['Score'] >= df['Score'].quantile(0.5)
                         and (row['Revenue ($)'] - row['Budget ($)']) / row['Budget ($)'] >= 0.5 else 'Unsuccessful', axis=1)
df

C:\Users\PC\AppData\Local\Temp\ipykernel_9884\4265024190.py:6: FutureWarning: using <class 'pandas.core.series.Series'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  genre_stack = df['Genre'].apply(splitGenre).agg(pd.Series).stack().agg(pd.Series).stack()
C:\Users\PC\AppData\Local\Temp\ipykernel_9884\4265024190.py:6: FutureWarning: using <class 'pandas.core.series.Series'> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  genre_stack = df['Genre'].apply(splitGenre).agg(pd.Series).stack().agg(pd.Series).stack()


,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Drama,Genre_Family,Genre_Fantasy,Genre_History,Genre_Horror,...,Genre_TV Movie,Genre_Thriller,Genre_War,Genre_Western,Runtime (h),Score,Budget ($),Revenue ($),Language_encoded,Success
0,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,2.37,72.0,340000000.0,704709660.0,5,Successful
1,0,0,1,1,0,0,1,1,0,0,...,0,0,0,0,1.53,72.0,95000000.0,173800000.0,5,Successful
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1.83,78.0,20000000.0,286700000.0,5,Successful
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,3.02,81.0,100000000.0,951000000.0,5,Successful
7,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1.80,65.0,40000000.0,8000000.0,5,Unsuccessful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4990,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1.62,56.0,9000000.0,146863.0,5,Unsuccessful
4992,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1.85,67.0,23000000.0,19652638.0,5,Unsuccessful
4995,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1.85,64.0,19500000.0,34854990.0,5,Unsuccessful
4996,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1.52,53.0,13000000.0,1784763.0,5,Unsuccessful


## Data preparation

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training model

In [5]:
# Train the SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Train the KNN model
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

# Train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Train the Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
''

''

## Prediction

In [6]:
y_pred_svm = svm_model.predict(X_test)
y_pred_knn = knn_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_lr = lr_model.predict(X_test)

## Evaluation

In [7]:
pos_label = 'Successful'

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, pos_label=pos_label)
recall_svm = recall_score(y_test, y_pred_svm, pos_label=pos_label)
f1_score_svm = f1_score(y_test, y_pred_svm, pos_label=pos_label)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn, pos_label=pos_label)
recall_knn = recall_score(y_test, y_pred_knn, pos_label=pos_label)
f1_score_knn = f1_score(y_test, y_pred_knn, pos_label=pos_label)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, pos_label=pos_label)
recall_rf = recall_score(y_test, y_pred_rf, pos_label=pos_label)
f1_score_rf = f1_score(y_test, y_pred_rf, pos_label=pos_label)

accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, pos_label=pos_label)
recall_lr = recall_score(y_test, y_pred_lr, pos_label=pos_label)
f1_score_lr = f1_score(y_test, y_pred_lr, pos_label=pos_label)

print("== SVM ==")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-score:", f1_score_svm)

print("== KNN ==")
print("Accuracy:", accuracy_knn)
print("Precision:", precision_knn)
print("Recall:", recall_knn)
print("F1-score:", f1_score_knn)

print("== Random Forest ==")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-score:", f1_score_rf)

print("== Logistic Regression ==")
print("Accuracy:", accuracy_lr)
print("Precision:", precision_lr)
print("Recall:", recall_lr)
print("F1-score:", f1_score_lr)

== SVM ==
Accuracy: 0.6750841750841751
Precision: 0.7055214723926381
Recall: 0.4423076923076923
F1-score: 0.5437352245862883
== KNN ==
Accuracy: 0.6734006734006734
Precision: 0.6447368421052632
Recall: 0.5653846153846154
F1-score: 0.6024590163934427
== Random Forest ==
Accuracy: 0.9713804713804713
Precision: 0.9418181818181818
Recall: 0.9961538461538462
F1-score: 0.9682242990654205
== Logistic Regression ==
Accuracy: 0.702020202020202
Precision: 0.6912442396313364
Recall: 0.5769230769230769
F1-score: 0.6289308176100629


## Results analysis

1.3.2


## Conclusion

**Your conclusion**